# toledowalleye - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 25, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [2]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [3]:
# connect to SQL Server.
SERVER = '34.206.73.189'  
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [4]:
lkupclientid = 52 # toledowalleye

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrAHL].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,52,524,1046716,2017,Full Season,28.06,1970-01-01,0.0,17,TRUE,730,503,6.52,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,38,None,0
1,52,633,1033171,2017,Full Season,28.06,1970-01-01,0.0,7,TRUE,730,128,12.75,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,38,None,0
2,52,918,1040709,2017,Partial Season,420.00,1970-01-01,0.0,39,TRUE,730,1571,15.53,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,38,None,0
3,52,1926,1046180,2017,Full Season,42.09,1970-01-01,0.0,0,TRUE,730,0,6.20,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,38,None,0
4,52,2895,1032119,2017,Partial Season,360.00,1970-01-01,0.0,15,TRUE,730,182,67.09,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,38,None,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7853 entries, 0 to 7852
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            7853 non-null   int64  
 1   dimCustomerMasterId     7853 non-null   int64  
 2   customerNumber          7851 non-null   object 
 3   year                    7853 non-null   int64  
 4   productGrouping         7853 non-null   object 
 5   totalSpent              7853 non-null   float64
 6   recentDate              7853 non-null   object 
 7   attendancePercent       7853 non-null   float64
 8   renewedBeforeDays       7853 non-null   int64  
 9   isBuyer                 7853 non-null   object 
 10  source_tenure           7853 non-null   int64  
 11  tenure                  7853 non-null   int64  
 12  distToVenue             7853 non-null   float64
 13  totalGames              7853 non-null   int64  
 14  recency                 7853 non-null   

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [6]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(7853, 19)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [7]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7853 entries, 0 to 7852
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    7853 non-null   int64  
 1   totalSpent              7853 non-null   float64
 2   attendancePercent       7853 non-null   float64
 3   source_tenure           7853 non-null   int64  
 4   totalGames              7853 non-null   int64  
 5   missed_games_1          7853 non-null   int64  
 6   missed_games_over_2     7853 non-null   int64  
 7   is_Lockdown             7853 non-null   int64  
 8   NumberofGamesPerSeason  7853 non-null   int64  
 9   isNextYear_Buyer        7853 non-null   int64  
dtypes: float64(2), int64(8)
memory usage: 613.6 KB


### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [8]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,year,totalSpent,attendancePercent,source_tenure,totalGames,missed_games_1,missed_games_over_2,is_Lockdown,NumberofGamesPerSeason,isNextYear_Buyer
year,1.00,0.01,0.83,-0.07,0.62,-0.27,-0.58,0.89,0.89,-0.56
totalSpent,0.01,1.00,0.05,0.08,0.25,-0.27,0.23,0.01,0.01,0.18
attendancePercent,0.83,0.05,1.00,-0.09,0.78,-0.24,-0.64,0.94,0.94,-0.37
source_tenure,-0.07,0.08,-0.09,1.00,-0.05,-0.13,0.18,-0.10,-0.10,0.10
totalGames,0.62,0.25,0.78,-0.05,1.00,-0.18,-0.48,0.70,0.70,-0.27
missed_games_1,-0.27,-0.27,-0.24,-0.13,-0.18,1.00,-0.43,-0.26,-0.26,-0.21
missed_games_over_2,-0.58,0.23,-0.64,0.18,-0.48,-0.43,1.00,-0.68,-0.68,0.53
is_Lockdown,0.89,0.01,0.94,-0.10,0.70,-0.26,-0.68,1.00,1.00,-0.39
NumberofGamesPerSeason,0.89,0.01,0.94,-0.10,0.70,-0.26,-0.68,1.00,1.00,-0.39
isNextYear_Buyer,-0.56,0.18,-0.37,0.10,-0.27,-0.21,0.53,-0.39,-0.39,1.00


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [9]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (6282, 10)
Unseen Data For Predictions: (1571, 10)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [10]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "totalSpent",
        "attendancePercent",
        "source_tenure",
        "totalGames",
        "missed_games_1",
        "missed_games_over_2",
        "is_Lockdown",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,1940
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(6282, 10)"
5,Missing Values,False
6,Numeric Features,8
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


({'lr': <pycaret.containers.models.classification.LogisticRegressionClassifierContainer at 0x7f9b32ba3a60>,
  'knn': <pycaret.containers.models.classification.KNeighborsClassifierContainer at 0x7f9b32ba3970>,
  'nb': <pycaret.containers.models.classification.GaussianNBClassifierContainer at 0x7f9b32ba3790>,
  'dt': <pycaret.containers.models.classification.DecisionTreeClassifierContainer at 0x7f9b32ba3a00>,
  'svm': <pycaret.containers.models.classification.SGDClassifierContainer at 0x7f9b32937f40>,
  'rbfsvm': <pycaret.containers.models.classification.SVCClassifierContainer at 0x7f9b32a22820>,
  'gpc': <pycaret.containers.models.classification.GaussianProcessClassifierContainer at 0x7f9b32a22940>,
  'mlp': <pycaret.containers.models.classification.MLPClassifierContainer at 0x7f9b32a22a00>,
  'ridge': <pycaret.containers.models.classification.RidgeClassifierContainer at 0x7f9b32d92fa0>,
  'rf': <pycaret.containers.models.classification.RandomForestClassifierContainer at 0x7f9b32b97160>

In [11]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9230,0.9653,0.9355,0.8090,0.8673,0.8135,0.8181,0.2800
lr,Logistic Regression,0.9214,0.9626,0.9659,0.7900,0.8688,0.8135,0.8223,0.3640


In [12]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9443,0.9706,0.9630,0.8497,0.9028,0.8640,0.8673
1,0.9225,0.9670,0.9481,0.8000,0.8678,0.8135,0.8194
2,0.9423,0.9764,0.9259,0.8681,0.8961,0.8562,0.8571
3,0.9225,0.9675,0.9185,0.8158,0.8641,0.8101,0.8129
4,0.9185,0.9620,0.9111,0.8092,0.8571,0.8004,0.8032
5,0.8984,0.9553,0.9104,0.7578,0.8271,0.7560,0.7624
6,0.9223,0.9642,0.9259,0.8117,0.8651,0.8108,0.8143
7,0.9323,0.9702,0.9556,0.8217,0.8836,0.8362,0.8409
8,0.9163,0.9659,0.9556,0.7818,0.8600,0.8012,0.8094
9,0.9104,0.9539,0.9407,0.7744,0.8495,0.7865,0.7941


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?